In [3]:
import ollama
import faiss
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import OllamaEmbeddings
from langchain.vectorstores import FAISS
import process_md as pm

In [6]:
def load_pdf(pdf_path):
    loader = PyPDFLoader(pdf_path)
    docs = loader.load()
    return docs

# Provide the path to your PDF file
# pdf_path = r"D:\Downloads\Chapter 4.pdf"  
pdf_path=r"pdfs/Chapter 4.pdf"
docs = load_pdf(pdf_path)
print(f"Loaded {len(docs)} document pages.")


Loaded 14 document pages.


In [5]:
# def chunk_text(docs, chunk_size=500, chunk_overlap=100):
#     text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
#     chunks = text_splitter.split_documents(docs)
#     return chunks

file_path = "data/journal.md"  
last_timestamp = "2025-01-08 08:30:00"  

file=pm.read_markdown(file_path)

text=pm.extract_new_content(file, last_known_timestamp=last_timestamp)

chunks =pm.split_into_chunks(text, chunk_size=500, overlap=100)

print(f"Split into {len(chunks)} text chunks.")


Split into 5 text chunks.


In [6]:
def store_in_vector_db(chunks):
    embeddings = OllamaEmbeddings(model="llama3.2:latest", base_url="http://localhost:11666")  
    db = FAISS.from_texts(chunks, embedding=embeddings)
    return db

db = store_in_vector_db(chunks)
print("FAISS vector store initialized with Llama3 embeddings.")


C:\Users\pniki\AppData\Local\Temp\ipykernel_15188\2136429511.py:2: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="llama3.2:latest", base_url="http://localhost:11666")


FAISS vector store initialized with Llama3 embeddings.


In [ ]:
def query_rag(db, query):
    docs = db.similarity_search(query, k=3)  # Retrieve top 3 relevant chunks
    context = "\n\n".join([doc.page_content for doc in docs])
    
    
    prompt = f"Answer the following question based on the context below:\n\nContext:\n{context}\n\nQuestion: {query}\n\nAnswer:"
    response = ollama.chat(model="llama3.2:latest", messages=[{"role": "user", "content": prompt}])
    
    return response["message"]["content"]

query = "This is a document of my rants to myself. There are pecific timestamps with each message. From this information help me write a summary of each day for my personal journal. Make sure to preserve the emotional information and make sure the writing is in first person"
# query = "What does the document say about the president of the united sttates?"
response = query_rag(db, query)

print("RAG Response:", response)


RAG Response: The document appears to be a personal journal or diary written by someone who is a graduate student in academia, likely working on a PhD project involving artificial intelligence (AI) and machine learning. The entries are brief and humorous, detailing the author's struggles with their research, frustration with academic requirements, and feelings of uncertainty about their career path.

The writer seems to be struggling with various aspects of their PhD journey, including:

* Confusion about their research direction
* Difficulty understanding complex concepts, such as SVM optimizations and deep learning interpretability
* Frustration with debugging and data preprocessing issues
* Feeling overwhelmed by the amount of work and pressure to produce results
* Questioning whether they are making the right career choice

Despite these challenges, the writer also expresses moments of inspiration, creativity, and humor, such as watching cat videos or attending a talk on explainabl